In [3]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import mplfinance as mpf
import time


In [4]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()


In [5]:
keyspace_name = 'stock_data'

session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [6]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- stock_technical
- system_auth
- system_schema
- system_distributed
- system
- system_traces
- stock_data


In [7]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
    app_id="ZGCsmM0t5c9VMLRJ",                                 
    app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Hilman-D")
account_info = deri.get_account_info()

account_info 

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [8]:
# Get real-time quote
market_data = investor.MarketData()
stock = market_data.get_quote_symbol("AOT")
print(stock)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': 41.75, 'low': 40.25, 'last': 41.25, 'average': 41.07609, 'change': 1.25, 'percentChange': 3.13, 'totalVolume': 34500, 'totalBuyVolume': 23400, 'totalSellVolume': 10900, 'totalNoSideVolume': 200, 'status': '', 'marketStatus': 'OffHour', 'securityType': 'CS', 'eps': 0.37, 'pe': 26.83, 'pbv': 4.14, 'percentYield': 2.109999895095825, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [9]:
# get candlestick
market_cd = investor.MarketData()
res = market_cd.get_candlestick(symbol="AOT", interval="1d",normalized=True,)

In [10]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,114622,1682269200,72.00,72.75,72.00,72.75,13050472,9.451594e+08
1,114622,1682355600,72.50,72.75,72.25,72.25,15141138,1.098014e+09
2,114622,1682442000,72.50,73.00,72.50,73.00,10924024,7.954887e+08
3,114622,1682528400,73.25,73.50,72.75,73.00,12732294,9.303291e+08
4,114622,1682614800,73.00,73.25,72.75,73.25,15295296,1.118574e+09
...,...,...,...,...,...,...,...,...
495,114622,1746550800,37.25,39.50,37.25,39.25,57889130,2.237424e+09
496,114622,1746637200,39.00,39.25,38.50,38.50,33155908,1.284850e+09
497,114622,1746723600,39.00,39.25,38.25,39.00,27097430,1.048902e+09
498,114622,1747069200,39.75,39.75,37.50,37.50,48685413,1.856423e+09


In [11]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [12]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()

In [27]:
session.set_keyspace('stock_data')

In [28]:
insert_query = session.prepare("""
    INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""")


In [29]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': 41.75, 'low': 40.25, 'last': 41.25, 'average': 41.07609, 'change': 1.25, 'percentChange': 3.13, 'totalVolume': 34500, 'totalBuyVolume': 23400, 'totalSellVolume': 10900, 'totalNoSideVolume': 200, 'status': '', 'marketStatus': 'OffHour', 'securityType': 'CS', 'eps': 0.37, 'pe': 26.83, 'pbv': 4.14, 'percentYield': 2.109999895095825, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [30]:
from datetime import datetime

def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception:
        return None

try:
    session.execute(insert_query, (
        res.get("symbol"),
        datetime.now().date(),  # ใช้วันที่ปัจจุบันหรือกำหนดเอง
        res.get("open") if res.get("open") is not None else res.get("last"),
        res.get("high"),
        res.get("low"),
        res.get("close") if res.get("close") is not None else res.get("last"),
        int(res.get("totalVolume")) if res.get("totalVolume") is not None else 0
    ))
    print("✅ Inserted AOT data successfully.")
except Exception as e:
    print(f"❌ Error inserting AOT data: {e}")


✅ Inserted AOT data successfully.


In [31]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name} LIMIT 10")
df = pd.DataFrame(rows)
df.head(10)

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,NaN,NaN,NaN,NaN,0
1,TPCH,2025-05-16,NaN,NaN,NaN,NaN,0
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,NaN,NaN,NaN,NaN,0
4,QHBREIT,2025-05-16,NaN,NaN,NaN,NaN,0
5,VCOM,1970-01-01,2.4,2.44,2.38,2.44,152327
6,VCOM,2025-05-16,NaN,NaN,NaN,NaN,0
7,KDH,2025-05-16,NaN,NaN,NaN,NaN,0
8,NVD,2025-05-16,NaN,NaN,NaN,NaN,0
9,JDF,2025-05-16,NaN,NaN,NaN,NaN,0


In [18]:
def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception:
        return None

def insert_technical_data(symbol, market_data=market_data, session=session):
    print(f"🔍 กำลังดึงข้อมูล {symbol} ...")
    try:
        res = market_data.get_quote_symbol(symbol)
        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่พบข้อมูลสำหรับ {symbol}")
            return

        # Only use columns that exist in the table
        columns = ["symbol", "date", "open", "high", "low", "close", "volume"]
        # Prepare values for the columns
        values = [
            res.get("symbol"),
            datetime.now().date(),  # use current date
            res.get("open") if res.get("open") is not None else res.get("last"),
            res.get("high"),
            res.get("low"),
            res.get("close") if res.get("close") is not None else res.get("last"),
            int(res.get("totalVolume")) if res.get("totalVolume") is not None else 0
        ]

        insert_query = session.prepare("""
            INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """)
        session.execute(insert_query, values)
        print(f"✅ Insert สำเร็จ: {symbol}")
    except Exception as e:
        print(f"❌ Error: {symbol} — {e}")

# โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# Loop insert
for symbol in symbols:
    insert_technical_data(symbol)
    time.sleep(2)


🔍 กำลังดึงข้อมูล 24CS ...
✅ Insert สำเร็จ: 24CS


KeyboardInterrupt: 

In [19]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(1109, 7)

In [20]:
df.head()


,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,NaN,NaN,NaN,NaN,0
1,TPCH,2025-05-16,NaN,NaN,NaN,NaN,0
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,NaN,NaN,NaN,NaN,0
4,QHBREIT,2025-05-16,NaN,NaN,NaN,NaN,0


In [21]:
rows_symbol = session.execute(f"SELECT * FROM stock_data.stock_price WHERE symbol = 'ZIGA'")
df_symbol = pd.DataFrame(rows_symbol)
df_symbol.iloc[[1]]


,symbol,date,close,high,low,open,volume
1,ZIGA,2025-05-16,NaN,NaN,NaN,NaN,0


In [22]:
row = df_symbol.iloc[0]
non_null = row[row.notnull()]
print(non_null)

symbol          ZIGA
date      1970-01-01
close           1.47
high            1.49
low             1.33
open            1.34
volume      54135321
Name: 0, dtype: object


In [23]:
# ดึงข้อมูลเฉพาะคอลัมน์ที่ต้องการจาก Cassandra
rows_symbol_nonnull = session.execute(
    """
    SELECT symbol, date, open, high, low, close, volume
    FROM stock_data.stock_price
    """
)
# แปลงเป็น DataFrame
df_symbol_nonnull = pd.DataFrame(list(rows_symbol_nonnull))
# แสดงผลลัพธ์
df_symbol_nonnull.head()

,symbol,date,open,high,low,close,volume
0,PPPM,2025-05-16,NaN,NaN,NaN,NaN,0
1,TPCH,2025-05-16,NaN,NaN,NaN,NaN,0
2,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
3,POLY,2025-05-16,NaN,NaN,NaN,NaN,0
4,QHBREIT,2025-05-16,NaN,NaN,NaN,NaN,0
